# Merging data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data 

# opioid
opioid = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/opi_merge_final.parquet", engine='fastparquet')

# vital deaths
vital = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/Vital.parquet", engine='fastparquet')

# population
population = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/pop_final.parquet", engine='fastparquet')

In [3]:
opioid.sample(10)

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,MME
index,,,,,,,
4607,STILLWATER,MT,30095,MT,STILLWATER,2009,1.607280e+06
5607,WOODS,OK,40151,OK,WOODS,2009,2.189783e+06
496,RANDOLPH,AL,1111,AL,RANDOLPH,2007,5.209748e+06
7282,HUNT,TX,48231,TX,HUNT,2006,1.825012e+07
7353,JIM WELLS,TX,48249,TX,JIM WELLS,2014,1.040804e+07
8043,STARR,TX,48427,TX,STARR,2007,1.299432e+06
5526,SEMINOLE,OK,40133,OK,SEMINOLE,2009,1.131131e+07
8557,KLICKITAT,WA,53039,WA,KLICKITAT,2007,2.517741e+06
3124,GRANT,LA,22043,LA,GRANT,2013,4.443324e+06


In [4]:
vital.sample(10)

,County,County Code,Year,Deaths
3214,"Ingham County, MI",26065,2003,14
4569,"Miami County, OH",39109,2014,19
3509,"Kane County, IL",17089,2011,43
3647,"Kings County, NY",36047,2012,225
1780,"Dallas County, TX",48113,2013,260
2283,"Fairfield County, CT",9001,2014,100
5972,"Riverside County, CA",6065,2014,342
7710,"Williamson County, TX",48491,2006,23
2154,"El Paso County, TX",48141,2004,26
1589,"Columbus County, NC",37047,2014,10


In [5]:
population.sample(10)

,STATE,COUNTY,STNAME,CTYNAME,Year,FIPS
20696,29,85,Missouri,Hickory County,2009,9561
41404,55,35,Wisconsin,Eau Claire County,2015,102080
36382,26,57,Michigan,Gratiot County,2014,41467
28251,48,317,Texas,Martin County,2011,4884
110,4,25,Arizona,Yavapai County,2003,182090
28935,6,55,California,Napa County,2012,138538
17722,31,173,Nebraska,Thurston County,2008,6821
19004,54,21,West Virginia,Gilmer County,2008,8408
19,1,37,Alabama,Coosa County,2003,11732
23864,29,39,Missouri,Cedar County,2010,13982


In [6]:
population[population.duplicated(["FIPS", "Year"], keep=False)==True]

,STATE,COUNTY,STNAME,CTYNAME,Year,FIPS
19,1,37,Alabama,Coosa County,2003,11732
82,2,150,Alaska,Kodiak Island Borough,2003,13239
85,2,180,Alaska,Nome Census Area,2003,9210
92,2,261,Alaska,Valdez-Cordova Census Area,2003,9996
143,5,61,Arkansas,Howard County,2003,14098
...,...,...,...,...,...,...
41273,51,690,Virginia,Martinsville city,2015,13259
41334,54,7,West Virginia,Braxton County,2015,14375
41374,54,87,West Virginia,Roane County,2015,14375
41378,54,95,West Virginia,Tyler County,2015,8958


In [7]:
# Merge opioid and vital first

merge1 = pd.merge(opioid, vital, left_on=["fips", "year"], right_on=["County Code", "Year"], how='inner')

merge1

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,MME,County,County Code,Year,Deaths
0,BALDWIN,AL,1003,AL,BALDWIN,2006,5.103983e+07,"Baldwin County, AL",1003,2006,11
1,BALDWIN,AL,1003,AL,BALDWIN,2007,6.032305e+07,"Baldwin County, AL",1003,2007,24
2,BALDWIN,AL,1003,AL,BALDWIN,2008,6.682852e+07,"Baldwin County, AL",1003,2008,19
3,BALDWIN,AL,1003,AL,BALDWIN,2009,7.638538e+07,"Baldwin County, AL",1003,2009,29
4,BALDWIN,AL,1003,AL,BALDWIN,2010,9.272014e+07,"Baldwin County, AL",1003,2010,24
...,...,...,...,...,...,...,...,...,...,...,...
1775,YAKIMA,WA,53077,WA,YAKIMA,2010,6.746887e+07,"Yakima County, WA",53077,2010,24
1776,YAKIMA,WA,53077,WA,YAKIMA,2011,7.600735e+07,"Yakima County, WA",53077,2011,14
1777,YAKIMA,WA,53077,WA,YAKIMA,2012,8.023265e+07,"Yakima County, WA",53077,2012,25
1778,YAKIMA,WA,53077,WA,YAKIMA,2013,8.391631e+07,"Yakima County, WA",53077,2013,14


In [8]:
# Convert the fips column in merge1 data to strings and fill them to 5 digits. (for better merging with population data)

merge1["fips"] = merge1["fips"].astype(str)

merge1["fips"] = merge1["fips"].apply(lambda x: x.zfill(5))

merge1 = merge1.drop(columns=["year", "County Code"])

# Convert the year column in population data to int64 so that the type matches

population["Year"] = population["Year"].astype(int)
# population["FIPS"] = population["Year"].astype(int)

In [9]:
population['Year'].dtype

population['FIPS'].dtype

dtype('int64')

In [10]:
# Merge opioid and vital deaths data with population data, then

merge2 = pd.merge(merge1, population, left_on=["fips", "Year"], right_on=["FIPS", "Year"], how='inner')

merge_final = merge2[["BUYER_STATE", "BUYER_COUNTY", "Year", "MME", "Deaths", "Population", "fips"]]

merge_final["BUYER_STATE"].value_counts()


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
## Checking if we have same number of rows for all years. 
merge_final['Year'].value_counts()



NameError: name 'merge_final' is not defined